In [1]:
import pandas as pd
import pickle
import numpy as np
import gender_analysis
import matplotlib.pyplot as plt

fontsize = 30
plt.rcParams["font.size"] = fontsize
#plt.rcParams['font.family'] = "sans-serif"
plt.rcParams['xtick.direction'] = 'out' 
plt.rcParams['ytick.direction'] = 'out' 
plt.rcParams['xtick.major.width'] = 2.5 
plt.rcParams['xtick.minor.width'] = 1 
plt.rcParams['ytick.major.width'] = 2.5 
plt.rcParams['ytick.minor.width'] = 1 
plt.rcParams['axes.linewidth'] = 2.5 
plt.rcParams['xtick.major.size'] = 15
plt.rcParams['xtick.minor.size'] = 12.5
plt.rcParams['ytick.major.size'] = 15
plt.rcParams['ytick.minor.size'] = 12.5
plt.rcParams['hatch.linewidth'] = 0.3

pd.set_option("future.no_silent_downcasting", True)

In [2]:
# Parameters
orbis_test_size = 450
wgnd_test_size = 5000
random_state = 42
shuffle = True
include_middle_name = False
include_last_name = False
clearning_special_chars = True
min_ratio_train_to_test = 5
min_ratio_minority_to_majority = 0.2
min_name_samples = 500
arab_league = {
    "DZ", "BH", "KM", "DJ", "EG", "IQ", "JO", "KW", "LB", "LY", "MR", 
    "MA", "OM", "PS", "QA", "SA", "SO", "SD", "SY", "TN", "AE", "YE",
}

In [3]:
# Country selection
with open("./data/data_sets.pkl", 'rb') as f:
    data_by_country = pickle.load(f)

name_count_by_country = {c: (len(data_by_country[c][0]), len(data_by_country[c][1])) for c in data_by_country}

all_country_lst = [_[0] for _ in sorted(list(name_count_by_country.items()), key=lambda x: x[1], reverse=True)]
country_lst = []
n, n_ = 0, 0
arab_minority_group = set()
arab_name_samples = {}
for country in all_country_lst:
    f_name_count = len(data_by_country[country][0])
    m_name_count = len(data_by_country[country][1])
    
    if country in arab_league:
        arab_name_samples[country] = (f_name_count, m_name_count)
        
    f_test_size = orbis_test_size
    f_validation_size = orbis_test_size
    f_remaining_size = f_name_count - f_test_size - f_validation_size
    m_test_size = orbis_test_size
    m_validation_size = orbis_test_size
    m_remaining_size = m_name_count - m_test_size - m_validation_size
    
    if f_remaining_size <= m_remaining_size:
        f_train_size = f_remaining_size
        m_train_size = int(float(f_remaining_size) / min_ratio_minority_to_majority)
    else:
        f_train_size = int(float(f_remaining_size) / min_ratio_minority_to_majority)
        m_train_size = m_remaining_size
        
    print(country, f_name_count, f_test_size + f_validation_size, m_name_count, m_test_size + m_validation_size)

    if f_train_size < f_test_size * min_ratio_train_to_test or m_train_size < m_test_size * min_ratio_train_to_test:
        if country in arab_league and f_name_count > orbis_test_size and m_name_count > orbis_test_size:
            arab_minority_group.add(country)
        continue
        
    n_ += f_name_count + m_name_count
    country_lst.append(country)

country_lst += list(arab_minority_group)

countries_with_low_auc = {'CN', 'IN', 'BD', 'KR'}
    
f_name = "./data/complementnb_best_alpha.pickle"
with open(f_name, "rb") as f:
    nb_alpha = pickle.load(f)

f_name = "./data/complementnb_prob_threshold_in_orbis.pickle"
with open(f_name, "rb") as f:
    nb_prob_threshold_orbis = pickle.load(f)

f_name = "./data/complementnb_prob_threshold_in_wgnd.pickle"
with open(f_name, "rb") as f:
    nb_prob_threshold_wgnd = pickle.load(f)

nb_prob_threshold = {}
new_country_lst = []
for country in country_lst:
    if (country not in nb_prob_threshold_orbis or country not in nb_prob_threshold_wgnd 
            or country in countries_with_low_auc):
        continue
        
    p1 = nb_prob_threshold_orbis[country]
    p2 = nb_prob_threshold_wgnd[country]
    nb_prob_threshold[country] = max(p1, p2)
    new_country_lst.append(country)
    
country_lst = list(new_country_lst)
    
f_path = ("./data/author_gender"
          + "_middle_name_" + str(include_middle_name)
          + "_last_name_" + str(include_last_name)
          + "_special_chars_" + str(clearning_special_chars)
          + ".pkl")
with open(f_path, mode="rb") as f:
    author_gender = pickle.load(f)

f_path = ("./data/author_sample_lst"
          + "_middle_name_" + str(include_middle_name)
          + "_last_name_" + str(include_last_name)
          + "_special_chars_" + str(clearning_special_chars)
          + ".pkl")

with open(f_path, mode="rb") as f:
    (author_name_corpus, author_sample_lst) = pickle.load(f)
    
num_authors_by_country = {}
for country in country_lst:

    n_f, n_m = 0, 0
    for author_id in author_sample_lst[country]:
        g = author_gender.get(author_id, -1)
        if g == 0:
            n_f += 1
        elif g == 1:
            n_m += 1

    num_authors_by_country[country] = [n_f, n_m]
    
new_country_lst_ = []
for country in country_lst:
    if (country not in nb_prob_threshold_orbis or country not in nb_prob_threshold_wgnd 
            or country in countries_with_low_auc):
        continue
        
    n_f = num_authors_by_country[country][0]
    n_m = num_authors_by_country[country][1]
    
    if n_f < 1000 or n_m < 1000:
        continue
        
    new_country_lst_.append(country)

country_lst = list(new_country_lst_)

print(len(new_country_lst), len(new_country_lst_))
print(set(new_country_lst) - set(new_country_lst_))
print(new_country_lst_)

RU 6809103 900 10065354 900
GB 4736468 900 7911337 900
ZA 3061224 900 3829995 900
IT 2508971 900 5349306 900
AR 1732553 900 2194551 900
UA 1307747 900 2329730 900
FI 616804 900 824248 900
RO 612856 900 949509 900
BG 610126 900 823619 900
NO 426803 900 695419 900
CH 377475 900 747701 900
NZ 315754 900 518050 900
RS 174991 900 331788 900
MY 164274 900 501751 900
CZ 154167 900 305160 900
CL 148767 900 324038 900
TH 110090 900 305600 900
IN 95120 900 306644 900
CN 91873 900 223318 900
VN 87210 900 394714 900
DE 83696 900 270400 900
JO 72867 900 409478 900
IE 71659 900 150820 900
US 70970 900 260674 900
FR 62720 900 171151 900
PH 62400 900 41599 900
PL 56743 900 134037 900
LB 56421 900 131020 900
AU 46689 900 105329 900
PG 36712 900 115424 900
BH 35966 900 45900 900
OM 34553 900 140942 900
PK 29193 900 154307 900
DZ 28313 900 295427 900
MM 27661 900 78479 900
NG 27310 900 56344 900
SK 25059 900 53108 900
SE 25025 900 63070 900
ES 23916 900 60953 900
ID 23763 900 65262 900
HK 23506 900 67050

In [4]:
gender_analysis.calc_women_rate_by_publication_year_and_country(country_lst, include_middle_name, include_last_name, clearning_special_chars)

{'RU': {1950: [8, 61],
  1951: [8, 50],
  1952: [11, 68],
  1953: [11, 79],
  1954: [24, 107],
  1955: [11, 107],
  1956: [40, 214],
  1957: [45, 156],
  1958: [63, 224],
  1959: [69, 268],
  1960: [69, 316],
  1961: [100, 397],
  1962: [129, 554],
  1963: [173, 675],
  1964: [217, 810],
  1965: [240, 880],
  1966: [240, 975],
  1967: [274, 1158],
  1968: [304, 1127],
  1969: [312, 1310],
  1970: [359, 1469],
  1971: [347, 1429],
  1972: [414, 1717],
  1973: [386, 1701],
  1974: [378, 1717],
  1975: [425, 1829],
  1976: [422, 1870],
  1977: [409, 1902],
  1978: [451, 2016],
  1979: [485, 2085],
  1980: [547, 2179],
  1981: [556, 2312],
  1982: [552, 2286],
  1983: [585, 2360],
  1984: [626, 2655],
  1985: [661, 2697],
  1986: [620, 2597],
  1987: [689, 2787],
  1988: [706, 3128],
  1989: [913, 3764],
  1990: [940, 4360],
  1991: [1189, 4966],
  1992: [1158, 4907],
  1993: [1218, 4913],
  1994: [1434, 5525],
  1995: [1493, 5673],
  1996: [1678, 6451],
  1997: [1832, 6707],
  1998: [2136

In [5]:
survival_probability = {}
pub_interval_threshold = {}

for target_country in country_lst:
    print(target_country)

    (female_survival_probability, male_survival_probability) = gender_analysis.calc_survival_probability_for_publication_interval(target_country, include_middle_name,include_last_name, clearning_special_chars)
    
    survival_probability[target_country] = (female_survival_probability, male_survival_probability)
    
    pub_interval_threshold[target_country] = gender_analysis.calc_threshold_for_publication_interval(female_survival_probability, male_survival_probability)
    
    print([float(d_int)/365 for d_int in pub_interval_threshold[target_country]])
    
f_path = ("./data/survival_probability"
              + "_middle_name_" + str(include_middle_name)
              + "_last_name_" + str(include_last_name)
              + "_special_chars_" + str(clearning_special_chars)
              + ".pkl")

with open(f_path, "wb") as f:
    pickle.dump(survival_probability, f)
    
f_path = ("./data/pub_interval_threshold"
              + "_middle_name_" + str(include_middle_name)
              + "_last_name_" + str(include_last_name)
              + "_special_chars_" + str(clearning_special_chars)
              + ".pkl")

with open(f_path, "wb") as f:
    pickle.dump(pub_interval_threshold, f)

RU
[11.243835616438357, 12.01095890410959]
GB
[14.594520547945205, 12.378082191780821]
ZA
[13.526027397260274, 12.597260273972603]
IT
[8.756164383561643, 7.923287671232877]
RO
[9.926027397260274, 9.345205479452055]
BG
[13.347945205479451, 14.70958904109589]
NO
[10.150684931506849, 9.742465753424657]
CH
[10.860273972602739, 9.865753424657534]
NZ
[14.509589041095891, 12.095890410958905]
RS
[10.01095890410959, 10.150684931506849]
CZ
[8.676712328767124, 8.594520547945205]
DE
[11.008219178082191, 9.610958904109589]
JO
[9.797260273972602, 9.838356164383562]
IE
[15.687671232876712, 13.591780821917808]
US
[13.56986301369863, 12.347945205479451]
FR
[10.008219178082191, 9.391780821917807]
PH
[15.361643835616439, 15.013698630136986]
PL
[9.008219178082191, 8.90958904109589]
LB
[13.07123287671233, 12.893150684931507]
AU
[11.797260273972602, 10.427397260273972]
BH
[16.013698630136986, 15.92876712328767]
OM
[11.005479452054795, 9.69041095890411]
DZ
[10.345205479452055, 10.01095890410959]
NG
[11.17534

In [6]:
# All countries

f_path = ("./data/pub_interval_threshold"
              + "_middle_name_" + str(include_middle_name)
              + "_last_name_" + str(include_last_name)
              + "_special_chars_" + str(clearning_special_chars)
              + ".pkl")
with open(f_path, mode="rb") as f:
    pub_interval_threshold = pickle.load(f)

productive_people_pyramid = {}

for target_country in country_lst:
    print(target_country)
    
    target_years = [1970, 1980, 1990, 2000, 2010, 2023]
    
    for target_year in target_years:
        female_productivity, male_productivity, female_count, male_count = gender_analysis.calc_productive_people_pyramid(target_country, include_middle_name,include_last_name, clearning_special_chars, pub_interval_threshold[target_country], target_year)
        productive_people_pyramid[(target_country, target_year)] = (female_count, male_count)
    
    base_year, target_year = 2010, 2100
    productive_people_pyramid[(target_country, base_year, target_year)] = gender_analysis.calc_future_productive_people_pyramid(target_country, include_middle_name,include_last_name, clearning_special_chars, pub_interval_threshold[target_country], base_year, target_year)
    
    base_year, target_year = 2023, 2100
    productive_people_pyramid[(target_country, base_year, target_year)] = gender_analysis.calc_future_productive_people_pyramid(target_country, include_middle_name,include_last_name, clearning_special_chars, pub_interval_threshold[target_country], base_year, target_year)
    
f_path = ("./data/productive_people_pyramid"
              + "_middle_name_" + str(include_middle_name)
              + "_last_name_" + str(include_last_name)
              + "_special_chars_" + str(clearning_special_chars)
              + ".pkl")

with open(f_path, "wb") as f:
    pickle.dump(productive_people_pyramid, f)

RU
GB
ZA
IT
RO
BG
NO
CH
NZ
RS
CZ
DE
JO
IE
US
FR
PH
PL
LB
AU
BH
OM
DZ
NG
SE
ES
IS
NL
MD
EG
CA
PT
TR
KE
KW
LT
AT
DK
AE
JP
ZW
BE
EE
IR
ME
SA
LK
JM
GH
AL
BY
IL
MA
BA
CY
TN
SY
IQ
